In [2]:
# Example usage
csv_files = [
    "/home/sur06423/project/Version_5_exp/YOLO_11/v11_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_11/v11_s/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_11/v11_n/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_s/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_n/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_s/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_n/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_s/train5/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_n/train/results.csv",
]



In [8]:
import pandas as pd
file_path = "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_n/train/results.csv"
data = pd.read_csv(file_path)

In [9]:
print(data)

                   epoch        train/box_loss        train/cls_loss  \
0                      1               1.16030               2.79990   
1                      2               1.01150               1.36950   
2                      3               0.97131               1.19470   
3                      4               0.94840               1.07490   
4                      5               0.94123               0.96948   
..                   ...                   ...                   ...   
95                    96               0.71955               0.39834   
96                    97               0.71015               0.38745   
97                    98               0.71286               0.38875   
98                    99               0.69971               0.37950   
99                   100               0.69300               0.37665   

          train/dfl_loss  metrics/precision(B)     metrics/recall(B)  \
0                1.09260               0.99924               0.

In [10]:
data["    metrics/mAP50-95(B)"]

0     0.09381
1     0.17028
2     0.19608
3     0.19557
4     0.25755
       ...   
95    0.44287
96    0.44246
97    0.44157
98    0.44122
99    0.44293
Name:     metrics/mAP50-95(B), Length: 100, dtype: float64

In [18]:
data["       metrics/mAP50(B)"]

0     0.11329
1     0.24434
2     0.32293
3     0.32854
4     0.42036
       ...   
95    0.68012
96    0.68128
97    0.68293
98    0.67958
99    0.68179
Name:        metrics/mAP50(B), Length: 100, dtype: float64

In [19]:
data["                  epoch"]

0       1
1       2
2       3
3       4
4       5
     ... 
95     96
96     97
97     98
98     99
99    100
Name:                   epoch, Length: 100, dtype: int64

In [20]:
import pandas as pd

def process_single_result(file_path):
    try:
        # Read the CSV file
        data = pd.read_csv(file_path)

        # Strip whitespace from column names
        data.columns = data.columns.str.strip()

        # Standardize column names if needed
        column_mapping = {
            'metrics/mAP50(B)': 'mAP50',
            'metrics/mAP50-95(B)': 'mAP50-95',
            'epoch': 'epoch'
        }
        data.rename(columns=column_mapping, inplace=True)

        # Check if required columns exist
        required_columns = ['mAP50', 'mAP50-95', 'epoch']
        missing_columns = [col for col in required_columns if col not in data.columns]
        if missing_columns:
            raise KeyError(f"Missing columns in {file_path}: {missing_columns}")

        # Find the best epoch based on the first method
        best_epoch_method1 = data.iloc[data['mAP50-95'].idxmax()]
        method1_result = {
            "Epoch": best_epoch_method1['epoch'],
            "mAP50": best_epoch_method1['mAP50'],
            "mAP50-95": best_epoch_method1['mAP50-95']
        }

        # Find the best epoch based on the second method
        data['Fitness Score'] = 0.1 * data['mAP50'] + 0.9 * data['mAP50-95']
        best_epoch_method2 = data.iloc[data['Fitness Score'].idxmax()]
        method2_result = {
            "Epoch": best_epoch_method2['epoch'],
            "mAP50": best_epoch_method2['mAP50'],
            "mAP50-95": best_epoch_method2['mAP50-95'],
            "Fitness Score": best_epoch_method2['Fitness Score']
        }

        return method1_result, method2_result

    except KeyError as e:
        print(f"Error processing file {file_path}: {e}")
        return None, None
    except Exception as e:
        print(f"Unexpected error with file {file_path}: {e}")
        return None, None


In [21]:
# Example usage
file_path = "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_n/train/results.csv"
method1_result, method2_result = process_single_result(file_path)

In [22]:
# Display results
if method1_result:
    print("Best Epoch Based on mAP50-95 (Method 1):", method1_result)
if method2_result:
    print("Best Epoch Based on Fitness Score (Method 2):", method2_result)

Best Epoch Based on mAP50-95 (Method 1): {'Epoch': 100.0, 'mAP50': 0.68179, 'mAP50-95': 0.44293}
Best Epoch Based on Fitness Score (Method 2): {'Epoch': 100.0, 'mAP50': 0.68179, 'mAP50-95': 0.44293, 'Fitness Score': 0.466816}


In [23]:
import os
import pandas as pd

def process_results(files):
    method1_results = []
    method2_results = []

    for file_path in files:
        try:
            # Extract model name from the file path
            base_name = os.path.basename(file_path)
            dir_name = os.path.basename(os.path.dirname(file_path))
            model_name = f"{dir_name}_{base_name.split('_')[0]}"

            # Read the CSV file
            data = pd.read_csv(file_path)

            # Strip whitespace from column names
            data.columns = data.columns.str.strip()

            # Standardize column names if needed
            column_mapping = {
                'metrics/mAP50(B)': 'mAP50',
                'metrics/mAP50-95(B)': 'mAP50-95',
                'epoch': 'epoch'
            }
            data.rename(columns=column_mapping, inplace=True)

            # Check if required columns exist
            required_columns = ['mAP50', 'mAP50-95', 'epoch']
            missing_columns = [col for col in required_columns if col not in data.columns]
            if missing_columns:
                raise KeyError(f"Missing columns in {file_path}: {missing_columns}")

            # Find the best epoch based on the first method
            best_epoch_method1 = data.iloc[data['mAP50-95'].idxmax()]
            method1_results.append({
                "Model Name": model_name,
                "Epoch": best_epoch_method1['epoch'],
                "mAP50": best_epoch_method1['mAP50'],
                "mAP50-95": best_epoch_method1['mAP50-95']
            })

            # Find the best epoch based on the second method
            data['Fitness Score'] = 0.1 * data['mAP50'] + 0.9 * data['mAP50-95']
            best_epoch_method2 = data.iloc[data['Fitness Score'].idxmax()]
            method2_results.append({
                "Model Name": model_name,
                "Epoch": best_epoch_method2['epoch'],
                "mAP50": best_epoch_method2['mAP50'],
                "mAP50-95": best_epoch_method2['mAP50-95'],
                "Fitness Score": best_epoch_method2['Fitness Score']
            })

        except KeyError as e:
            print(f"Error processing file {file_path}: {e}")
        except Exception as e:
            print(f"Unexpected error with file {file_path}: {e}")

    # Create dataframes for both methods
    method1_df = pd.DataFrame(method1_results)
    method2_df = pd.DataFrame(method2_results)

    return method1_df, method2_df




In [24]:
# Example usage
csv_files = [
    "/home/sur06423/project/Version_5_exp/YOLO_11/v11_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_11/v11_s/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_11/v11_n/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_s/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_n/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_s/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_n/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_m/train/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_s/train5/results.csv",
    "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_n/train/results.csv",
]

In [25]:

method1_df, method2_df = process_results(csv_files)

# Display results
print("Best Epochs Based on mAP50-95 (Method 1):")
print(method1_df)

print("\nBest Epochs Based on Fitness Score (Method 2):")
print(method2_df)

Best Epochs Based on mAP50-95 (Method 1):
            Model Name  Epoch    mAP50  mAP50-95
0    train_results.csv   99.0  0.77148   0.52891
1    train_results.csv  100.0  0.73705   0.49936
2    train_results.csv   98.0  0.67776   0.43749
3    train_results.csv   99.0  0.75669   0.51826
4    train_results.csv   99.0  0.73679   0.49366
5    train_results.csv   99.0  0.66139   0.43328
6    train_results.csv   98.0  0.76316   0.52111
7    train_results.csv  100.0  0.72649   0.48589
8    train_results.csv   90.0  0.66651   0.43441
9    train_results.csv  100.0  0.77134   0.52571
10  train5_results.csv   97.0  0.73954   0.49606
11   train_results.csv  100.0  0.68179   0.44293

Best Epochs Based on Fitness Score (Method 2):
            Model Name  Epoch    mAP50  mAP50-95  Fitness Score
0    train_results.csv   99.0  0.77148   0.52891       0.553167
1    train_results.csv  100.0  0.73705   0.49936       0.523129
2    train_results.csv   98.0  0.67776   0.43749       0.461517
3    train_result

In [28]:
import os
import pandas as pd

def process_results(files):
    method1_results = []
    method2_results = []

    for file_path in files:
        try:
            # Extract model name from the file path
            base_name = os.path.basename(file_path)
            parent_dir = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            model_name = f"{parent_dir}_{os.path.basename(os.path.dirname(file_path))}"

            # Read the CSV file
            data = pd.read_csv(file_path)

            # Strip whitespace from column names
            data.columns = data.columns.str.strip()

            # Standardize column names if needed
            column_mapping = {
                'metrics/mAP50(B)': 'mAP50',
                'metrics/mAP50-95(B)': 'mAP50-95',
                'epoch': 'epoch'
            }
            data.rename(columns=column_mapping, inplace=True)

            # Check if required columns exist
            required_columns = ['mAP50', 'mAP50-95', 'epoch']
            missing_columns = [col for col in required_columns if col not in data.columns]
            if missing_columns:
                raise KeyError(f"Missing columns in {file_path}: {missing_columns}")

            # Find the best epoch based on the first method
            best_epoch_method1 = data.iloc[data['mAP50-95'].idxmax()]
            method1_results.append({
                "Model Name": model_name,
                "Epoch": best_epoch_method1['epoch'],
                "mAP50": best_epoch_method1['mAP50'],
                "mAP50-95": best_epoch_method1['mAP50-95']
            })

            # Find the best epoch based on the second method
            data['Fitness Score'] = 0.1 * data['mAP50'] + 0.9 * data['mAP50-95']
            best_epoch_method2 = data.iloc[data['Fitness Score'].idxmax()]
            method2_results.append({
                "Model Name": model_name,
                "Epoch": best_epoch_method2['epoch'],
                "mAP50": best_epoch_method2['mAP50'],
                "mAP50-95": best_epoch_method2['mAP50-95'],
                "Fitness Score": best_epoch_method2['Fitness Score']
            })

        except KeyError as e:
            print(f"Error processing file {file_path}: {e}")
        except Exception as e:
            print(f"Unexpected error with file {file_path}: {e}")

    # Create dataframes for both methods
    method1_df = pd.DataFrame(method1_results)
    method2_df = pd.DataFrame(method2_results)

    return method1_df, method2_df




In [29]:

csv_files_mapping = {
    "YOLOv8n": "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_n/train/results.csv",
    "YOLOv8s": "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_s/train5/results.csv",
    "YOLOv8m":"/home/sur06423/project/Version_5_exp/YOLO_v8/v8_m/train/results.csv",
    "YOLOv9t": "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_n/train/results.csv",
    "YOLOv9s": "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_s/train/results.csv",
    "YOLOv9m":"/home/sur06423/project/Version_5_exp/YOLO_v9/v9_m/train/results.csv",
    "YOLOv10n":"/home/sur06423/project/Version_5_exp/YOLO_v10/v10_n/train/results.csv",
    "YOLOv10s":"/home/sur06423/project/Version_5_exp/YOLO_v10/v10_s/train/results.csv",
    "YOLOv10m":"/home/sur06423/project/Version_5_exp/YOLO_v10/v10_m/train/results.csv",
    "YOLO11n":"/home/sur06423/project/Version_5_exp/YOLO_11/v11_n/train/results.csv",
    "YOLO11s":"/home/sur06423/project/Version_5_exp/YOLO_11/v11_s/train/results.csv",
    "YOLO11m":"/home/sur06423/project/Version_5_exp/YOLO_11/v11_m/train/results.csv",
}

In [30]:

method1_df, method2_df = process_results(csv_files)

# Display results
print("Best Epochs Based on mAP50-95 (Method 1):")
print(method1_df)

print("\nBest Epochs Based on Fitness Score (Method 2):")
print(method2_df)

Best Epochs Based on mAP50-95 (Method 1):
     Model Name  Epoch    mAP50  mAP50-95
0    v8_n_train  100.0  0.68179   0.44293
1   v8_s_train5   97.0  0.73954   0.49606
2    v8_m_train  100.0  0.77134   0.52571
3    v9_n_train   90.0  0.66651   0.43441
4    v9_s_train  100.0  0.72649   0.48589
5    v9_m_train   98.0  0.76316   0.52111
6   v10_n_train   99.0  0.66139   0.43328
7   v10_s_train   99.0  0.73679   0.49366
8   v10_m_train   99.0  0.75669   0.51826
9   v11_n_train   98.0  0.67776   0.43749
10  v11_s_train  100.0  0.73705   0.49936
11  v11_m_train   99.0  0.77148   0.52891

Best Epochs Based on Fitness Score (Method 2):
     Model Name  Epoch    mAP50  mAP50-95  Fitness Score
0    v8_n_train  100.0  0.68179   0.44293       0.466816
1   v8_s_train5   97.0  0.73954   0.49606       0.520408
2    v8_m_train  100.0  0.77134   0.52571       0.550273
3    v9_n_train   90.0  0.66651   0.43441       0.457620
4    v9_s_train  100.0  0.72649   0.48589       0.509950
5    v9_m_train   98.0

In [32]:
import pandas as pd

def process_results(csv_files_mapping):
    method1_results = []
    method2_results = []

    for model_name, file_path in csv_files_mapping.items():
        try:
            # Read the CSV file
            data = pd.read_csv(file_path)

            # Strip whitespace from column names
            data.columns = data.columns.str.strip()

            # Standardize column names if needed
            column_mapping = {
                'metrics/mAP50(B)': 'mAP50',
                'metrics/mAP50-95(B)': 'mAP50-95',
                'metrics/precision(B)': 'Precision',
                'metrics/recall(B)': 'Recall',
                'epoch': 'epoch'
            }
            data.rename(columns=column_mapping, inplace=True)

            # Check if required columns exist
            required_columns = ['mAP50', 'mAP50-95', 'epoch', 'Precision', 'Recall']
            missing_columns = [col for col in required_columns if col not in data.columns]
            if missing_columns:
                raise KeyError(f"Missing columns in {file_path}: {missing_columns}")

            # Find the best epoch based on the first method
            best_epoch_method1 = data.iloc[data['mAP50-95'].idxmax()]
            method1_results.append({
                "Model Name": model_name,
                "Epoch": best_epoch_method1['epoch'],
                "Precision": best_epoch_method1['Precision'],
                "Recall": best_epoch_method1['Recall'],
                "mAP50": best_epoch_method1['mAP50'],
                "mAP50-95": best_epoch_method1['mAP50-95']
            })

            # Find the best epoch based on the second method
            data['Fitness Score'] = 0.1 * data['mAP50'] + 0.9 * data['mAP50-95']
            best_epoch_method2 = data.iloc[data['Fitness Score'].idxmax()]
            method2_results.append({
                "Model Name": model_name,
                "Epoch": best_epoch_method2['epoch'],
                "Precision": best_epoch_method2['Precision'],
                "Recall": best_epoch_method2['Recall'],
                "mAP50": best_epoch_method2['mAP50'],
                "mAP50-95": best_epoch_method2['mAP50-95'],
                "Fitness Score": best_epoch_method2['Fitness Score']
            })

        except KeyError as e:
            print(f"Error processing file {file_path}: {e}")
        except Exception as e:
            print(f"Unexpected error with file {file_path}: {e}")

    # Create dataframes for both methods
    method1_df = pd.DataFrame(method1_results)
    method2_df = pd.DataFrame(method2_results)

    return method1_df, method2_df

# Example usage
csv_files_mapping = {
    "YOLOv8n": "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_n/train/results.csv",
    "YOLOv8s": "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_s/train5/results.csv",
    "YOLOv8m": "/home/sur06423/project/Version_5_exp/YOLO_v8/v8_m/train/results.csv",
    "YOLOv9t": "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_n/train/results.csv",
    "YOLOv9s": "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_s/train/results.csv",
    "YOLOv9m": "/home/sur06423/project/Version_5_exp/YOLO_v9/v9_m/train/results.csv",
    "YOLOv10n": "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_n/train/results.csv",
    "YOLOv10s": "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_s/train/results.csv",
    "YOLOv10m": "/home/sur06423/project/Version_5_exp/YOLO_v10/v10_m/train/results.csv",
    "YOLO11n": "/home/sur06423/project/Version_5_exp/YOLO_11/v11_n/train/results.csv",
    "YOLO11s": "/home/sur06423/project/Version_5_exp/YOLO_11/v11_s/train/results.csv",
    "YOLO11m": "/home/sur06423/project/Version_5_exp/YOLO_11/v11_m/train/results.csv",
}

method1_df, method2_df = process_results(csv_files_mapping)

# Save results to CSV files
method1_df.to_csv("method1_results.csv", index=False)
method2_df.to_csv("method2_results.csv", index=False)

# Display results
print("Best Epochs Based on mAP50-95 (Method 1):")
print(method1_df)

print("\nBest Epochs Based on Fitness Score (Method 2):")
print(method2_df)


Best Epochs Based on mAP50-95 (Method 1):
   Model Name  Epoch  Precision   Recall    mAP50  mAP50-95
0     YOLOv8n  100.0    0.86877  0.60239  0.68179   0.44293
1     YOLOv8s   97.0    0.87040  0.65233  0.73954   0.49606
2     YOLOv8m  100.0    0.89172  0.67232  0.77134   0.52571
3     YOLOv9t   90.0    0.85681  0.59015  0.66651   0.43441
4     YOLOv9s  100.0    0.87516  0.64191  0.72649   0.48589
5     YOLOv9m   98.0    0.85315  0.68532  0.76316   0.52111
6    YOLOv10n   99.0    0.82104  0.58599  0.66139   0.43328
7    YOLOv10s   99.0    0.86741  0.64925  0.73679   0.49366
8    YOLOv10m   99.0    0.85600  0.66572  0.75669   0.51826
9     YOLO11n   98.0    0.87111  0.59615  0.67776   0.43749
10    YOLO11s  100.0    0.88900  0.64609  0.73705   0.49936
11    YOLO11m   99.0    0.89441  0.66853  0.77148   0.52891

Best Epochs Based on Fitness Score (Method 2):
   Model Name  Epoch  Precision   Recall    mAP50  mAP50-95  Fitness Score
0     YOLOv8n  100.0    0.86877  0.60239  0.68179   0.4